In [28]:
import numpy as np
import pandas as pd
import random
import os

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [29]:
train_y = train['Income']
train_X = train.drop(columns=['Income', 'ID'])

test_X = test.drop(columns=['ID'])

In [4]:
from sklearn.preprocessing import LabelEncoder

encoding_list = train.dtypes[train.dtypes == 'object'].index.tolist()
encoding_list.remove('ID')

for i in encoding_list:
    le = LabelEncoder()

    train_X[i] = train_X[i].astype(str)
    test_X[i] = test_X[i].astype(str)

    le.fit(train_X[i])
    train_X[i] = le.transform(train_X[i])

    for case in np.unique(test_X[i]):
        if case not in le.classes_:
            le.classes_ = np.append(le.classes_, case)
    
    test_X[i] = le.transform(test_X[i])


In [19]:
from sklearn.preprocessing import OneHotEncoder

encoding_list = train.dtypes[train.dtypes == 'object'].index.tolist()
encoding_list.remove('ID')

for i in encoding_list:

    items = np.array(train_X[i]).reshape(-1, 1)
    encoder = OneHotEncoder()
    encoder.fit(items)
    oh_label = encoder.transform(items)
    


    train_X[i] = train_X[i].astype(str)
    test_X[i] = test_X[i].astype(str)
    print(train_X[i])
    encoder.fit(train_X[i])
    encoder.transform(train_X[i]).toarray()


    print(encoder)
    # cats_enc = pd.DataFrame(oh_enc, 
    #                         columns=encoder.get_feature_names(i), 
    #                         dtype=np.int64)

0        M
1        M
2        F
3        M
4        F
        ..
19995    M
19996    F
19997    M
19998    F
19999    F
Name: Gender, Length: 20000, dtype: object


ValueError: Expected 2D array, got 1D array instead:
array=['M' 'M' 'F' ... 'M' 'F' 'F'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [24]:
# 더 쉽게 구현하기 get_dummies(items)
import pandas as pd

# DataFrame을 만들어 줄 때 딕셔너리 형태로 넣어주기
for i in encoding_list:
    df = pd.DataFrame(train_X[i])
    encoding_df = pd.get_dummies(df)
    encoding_df = encoding_df.astype(int)
    print(encoding_df) 

       Gender_F  Gender_M
0             0         1
1             0         1
2             1         0
3             0         1
4             1         0
...         ...       ...
19995         0         1
19996         1         0
19997         0         1
19998         1         0
19999         1         0

[20000 rows x 2 columns]
       Education_Status_Associates degree (Academic)   
0                                                  0  \
1                                                  0   
2                                                  0   
3                                                  0   
4                                                  0   
...                                              ...   
19995                                              0   
19996                                              0   
19997                                              0   
19998                                              0   
19999                                         

In [30]:

from catboost import CatBoostRegressor

# CatBoost 모델 초기화
cb = CatBoostRegressor()

# CatBoost 모델 학습
cb.fit(train_X, train_y)

# 테스트 데이터에 대한 예측
pred_y = cb.predict(test_X)

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=1]="M": Cannot convert 'b'M'' to float

In [8]:
submission = pd.read_csv('../data/sample_submission.csv')
submission['Income'] = pred_y
submission

,ID,Income
0,TEST_0000,-19.722083
1,TEST_0001,-28.980844
2,TEST_0002,402.812927
3,TEST_0003,713.170120
4,TEST_0004,-9.295254
...,...,...
9995,TEST_9995,873.861117
9996,TEST_9996,796.635309
9997,TEST_9997,344.934564
9998,TEST_9998,3.687604


In [ ]:
submission.to_csv('../output/cat_submission.csv', index=False)